In [15]:
import numpy as np
from environment.gobang import WrapperGoBang
from agent import Agent
import random
import os
from loguru import logger

In [16]:
class GoBangRandomAgent(Agent):
    def step(self, obs, explore=True, legal_actions=None):
        return {'action': random.choice(legal_actions)}

In [17]:
def do_exp(agent, gobang_n, epochs, test_n, test_freq, result_path):
    random_agent = GoBangRandomAgent(None)
    loss_episode = []
    reward_episode = []
    win_ratio = []
    best_win_ratio = -np.inf
    env = WrapperGoBang(gobang_n)
    for i in range(epochs):
        loss_list, reward_sum = agent.train(env)
        logger.info(f'Episode {i}/{epochs}, average loss: {np.mean(loss_list) if loss_list else -1}, reward sum: {reward_sum}')
        if loss_list:
            loss_episode.append(np.array([i, np.mean(loss_list)]))
            reward_episode.append(np.array([i, reward_sum]))
        
        if i % test_freq == 0:
            win = 0
            for _ in range(test_n):
                win += agent.test(env, random_agent, display=False)
            logger.info(f'Episode {i}/{epochs}, win {win}/{test_n}')
            ratio = 1.0 * win / test_n
            win_ratio.append(np.array([i, ratio]))
            if ratio > best_win_ratio:
                agent.save()
    np.save(os.path.join(result_path, 'loss.npy'), np.array(loss_episode))
    np.save(os.path.join(result_path, 'reward.npy'), np.array(reward_episode))
    np.save(os.path.join(result_path, 'ratio.npy'), np.array(win_ratio))    
    

In [18]:
from agent import DQNAgent
from agent.model import ConvQModel
from agent.configuration import DQNConfig
from agent.exploration import LinearExploration

In [19]:
eval_model = ConvQModel(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=32,
    output_dim=49,
    dueling=False
)
target_model = ConvQModel(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=32,
    output_dim=49,
    dueling=False
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = DQNConfig(
    eval_model=eval_model,
    target_model=target_model,
    ckpt_path='./ddqn_gobang_n7.ckpt',
    batch_size=32,
    lr=0.001,
    gamma=0.99,
    target_replace_frequency=100,
    capacity=5000,
    max_grad_norm=1.0,
    exploration=exploration,
    ddqn=True
)
agent = DQNAgent(config)
do_exp(agent, 7, 5000, 50, 50, './experiment/ddqn_gobang_n7')

2023-05-05 04:45:51.361 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: 0.19211230033553583, reward sum: -42
2023-05-05 04:46:09.853 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 22/50
2023-05-05 04:46:12.714 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 0.03307845235602596, reward sum: -46
2023-05-05 04:46:15.661 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 0.04470201044550746, reward sum: -45
2023-05-05 04:46:18.451 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 0.05145311639247476, reward sum: -46
2023-05-05 04:46:21.032 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: 0.011742640193423613, reward sum: -40
2023-05-05 04:46:23.139 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 0.07784668722516602, reward sum: -33
2023-05-05 04:46:25.941 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 0.023022174529769412, reward sum: -38
2023-05-05 04:46:28.708 | INFO     | __

In [20]:
eval_model = ConvQModel(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=32,
    output_dim=121,
    dueling=False
)
target_model = ConvQModel(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=32,
    output_dim=121,
    dueling=False
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = DQNConfig(
    eval_model=eval_model,
    target_model=target_model,
    ckpt_path='./ddqn_gobang_n11.ckpt',
    batch_size=32,
    lr=0.001,
    gamma=0.99,
    target_replace_frequency=100,
    capacity=5000,
    max_grad_norm=1.0,
    exploration=exploration,
    ddqn=True
)
agent = DQNAgent(config)
do_exp(agent, 11, 5000, 50, 50, './experiment/ddqn_gobang_n11')

2023-05-05 05:11:13.604 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: 0.0821610350376101, reward sum: -79
2023-05-05 05:11:45.255 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 37/50
2023-05-05 05:11:47.055 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 0.04319498497599453, reward sum: -86
2023-05-05 05:11:49.112 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 0.04171183745714246, reward sum: -77
2023-05-05 05:11:51.377 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 0.04476116458313168, reward sum: -90
2023-05-05 05:11:52.742 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: 0.056313858928314475, reward sum: -48
2023-05-05 05:11:55.239 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 0.04092618440268358, reward sum: -89
2023-05-05 05:11:57.726 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 0.03758859669752395, reward sum: -92
2023-05-05 05:11:59.881 | INFO     | __ma